In [1]:
import spacy
import pandas as pd
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
print('done')

def doDataScience(review):
    url = "https://raw.githubusercontent.com/MedicinalCabinet/DataScience/master/Data/cannabis.csv"
    df = pd.read_csv(url)
    df1 = df.dropna()
    label = df1[df1["Description"]=="None"].index
    
    #Drop all rows that have none values in all rows
    df1 = df1.drop(index=label)
    #import spacy.cli
    nlp = spacy.load("en_core_web_lg")
    # create tokenizer object
    tokenizer = Tokenizer(nlp.vocab)
    
    def tokenize(doc):
        """Return the tokens"""
        return [token.text for token in tokenizer(doc)]

    def get_lemmas(text):
        """Return the Lemmas"""
        lemmas = []
        doc = nlp(text)
    
        for token in doc: 
            if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
                lemmas.append(token.lemma_)
        return lemmas

    #combined the column Fffects, alltext, flavor
    df1['alltext'] = df1['Effects'].str.cat(df1["Flavor"], sep=" ")
    df1['alltext'] = df1['alltext'].str.cat(df1["Description"], sep=" ")

    text = df1["alltext"]

    # Instantiate vectorizer object
    tfidf = TfidfVectorizer(tokenizer=get_lemmas, min_df=0.025, max_df=.98, ngram_range=(1,2))

    # Create a vocabulary and get word counts per document
    dtm = tfidf.fit_transform(text) # Similiar to fit_predict

    # Get feature names to use as dataframe column headers
    dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

    # Fit on TF-IDF Vectors
    nn  = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
    nn.fit(dtm)
   

    new = tfidf.transform(review)
    nn.kneighbors(new.todense())
    array =nn.kneighbors(new.todense())
    whatIndex = array[1][0][0].astype(int)
    
    return df1['Description'][whatIndex]

done


In [2]:
input = "['sweet', 'tangy', 'strong']"
input = [input]
print(doDataScience(input))
print('done')

Red Poison, bred by Sweet Seeds, is an auto-flowering hybrid of Green Poison and a Pakistani landrace strain, believed to be a Hindu Kush phenotype that was selected for its deep purple colors. The influence of cannabis ruderalis in Red Poison’s genetics induces the flowering cycle automatically and helps this strain complete its lifecycle, from seed to harvest, in about 8 weeks. The buds of Red Poison are almost too pretty to consume with their amazing range of dark red and violet hues. The aroma is a sweet mixture of fruit flavors and skunky musk that bring with it balanced hybrid effects that relax the body and blanket your head in a cloud of euphoria.
done
